# Inspect/Visualise CNFM-E fit to motion corrected data

This notebook is used mainly to curate the components found by CNMF-E. 
The first curation steps are automated and use some of caiman's code, after that is manual curation where components can be merged or eliminated.  

In [1]:
try:
    get_ipython().magic(u'load_ext autoreload')
    get_ipython().magic(u'autoreload 2')
    get_ipython().magic(u'matplotlib qt')
except:
    pass

import matplotlib.pyplot as plt
import numpy as np
import os
from tqdm import tqdm
import napari


import caiman as cm
from caiman.source_extraction.cnmf.spatial import threshold_components
from caiman.utils.visualization import inspect_correlation_pnr

import logging
from fancylog import fancylog
import fancylog as package

import cv2
try:
    cv2.setNumThreads(0)
except:
    pass
import bokeh.plotting as bpl

from fcutils.file_io.io import load_yaml
from fcutils.video.utils import open_cvwriter, get_cap_from_images_folder, save_videocap_to_video
from fcutils.plotting.utils import clean_axes, save_figure, add_colorbar_to_img

from utils import print_cnmfe_components, plot_components_over_image, load_fit_cnmfe
from utils import start_server, load_params,  log_cnmfe_components, load_mmap_video_caiman

bpl.output_notebook()
c, dview, n_processes = start_server()

Loading BokehJS ...

## Load metadata, model and data

In [2]:
fld = 'D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\BF164p2\\19JUN26'
metadata = load_yaml(os.path.join(fld, "01_PARAMS", "analysis_metadata.yml"))

# Load video
video = os.path.join(fld, metadata[metadata['video_for_cnmfe_fit']])
Yr, dims, T, images = load_mmap_video_caiman(video)
bg = np.max(images, axis=0)


# Load model
cnm, model_filepath = load_fit_cnmfe(fld, n_processes, dview, raw=True)

# Start logging
fancylog.start_logging(os.path.join(fld, "02_LOGS"), package, file_log_level="INFO", variables=[cnm.params], verbose=True,    filename='cnmfe_component_curation_logs')
logging.info("Starting CNMF-E component inspection")
logging.info(f"Loading CNMF-E model from: {model_filepath}")

# Add analysis metadata to log
logging.info("ANALYSIS METADATA FILE:")
for k,v in metadata.items():
    logging.info(f"{k}: {v}")

logging.info(f"Video used for CNMF-E fitting: {metadata['video_for_cnmfe_fit']}: {video}")


INFO:root:Starting logging
2020-04-23 17:07:23 PM - INFO - MainProcess fancylog.py:271 - Starting logging
INFO:root:Multiprocessing-logging module not found, not logging multiple processes.
2020-04-23 17:07:23 PM - INFO - MainProcess fancylog.py:273 - Multiprocessing-logging module not found, not logging multiple processes.
INFO:root:Starting CNMF-E component inspection
2020-04-23 17:07:23 PM - INFO - MainProcess <ipython-input-2-de0798bbd091>:15 - Starting CNMF-E component inspection
INFO:root:Loading CNMF-E model from: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\cnmfe_fit_raw.hdf5
2020-04-23 17:07:23 PM - INFO - MainProcess <ipython-input-2-de0798bbd091>:16 - Loading CNMF-E model from: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\cnmfe_fit_raw.hdf5
INFO:root:ANALYSIS METADATA FILE:
2020-04-23 17:07:23 PM - INFO - MainProcess <ipython-input-2-de0798bbd091>:19 - ANALYSIS METADATA FILE:
INFO:root:experimenter: Federico
2020-04-23 17:07:

# Inspect components

Apply a number of quality controls of the components to keep only the good stuff.
Quality control steps include:

    - evaluate components 
    - threshold spatial components -> remove duplicates
    - remove small/large components [currently not implemented]

there's no undo for these operations, so if you need to restart from scratch you'll need
to re load cnm (or fit a new one even):

    cnm = load_fit_cnfm(model_path, n_processes, dview)

To check the params used for quality check use:

    cnm.params.quality

## Automatic quality evaluation

In [3]:
# Lets look at what things are like to start with
print_cnmfe_components(cnm, msg="Before quality control:")
log_cnmfe_components(cnm,  msg="Before quality control")

# Estimate
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)
coordinates = cm.utils.visualization.get_contours(cnm.estimates.A, bg.shape, thr=.2, thr_method="max")
good_compontents = cnm.estimates.idx_components

# log
print_cnmfe_components(cnm, msg="After evaluate components")
log_cnmfe_components(cnm,  msg="After evaluate components")

INFO:root:COMPONENTS - Before quality control
'Number of total components: 43
No accepted components yet

2020-04-23 17:07:24 PM - INFO - MainProcess utils.py:78 - COMPONENTS - Before quality control
'Number of total components: 43
No accepted components yet

INFO:root:Component evaluation in parallel
2020-04-23 17:07:24 PM - INFO - MainProcess components_evaluation.py:800 - Component evaluation in parallel


 ***** Before quality control:
Number of total components:  43
No accepted components yet

INFO:root:COMPONENTS - After evaluate components
'Number of total components: 43
Number of accepted components: 22
2020-04-23 17:07:38 PM - INFO - MainProcess utils.py:78 - COMPONENTS - After evaluate components
'Number of total components: 43
Number of accepted components: 22


 ***** After evaluate components
Number of total components:  43
Number of accepted components:  22


### Remove overlapping components
and components that are either too small ar too big

In [4]:
# Estimate spatial components
cnm.estimates.dims = dims
cnm.estimates.threshold_spatial_components(dview=dview)


if metadata['components_qc_params']['filter_by_size']:
    logging.info("Filtering components based on size")

    # load params
    _params = load_yaml(os.path.join(fld, "01_params", "params.yml"))['quality_evaluation_params']

    # Remove large/small and duplicate neurons
    cnm.estimates.remove_small_large_neurons(_params['min_size_neuro'], _params['max_size_neuro'])
    print_cnmfe_components(cnm, msg="After filter by size")
    log_cnmfe_components(cnm,  msg="After filter by size")

if metadata['components_qc_params']['remove_duplicate_components']:
    logging.info("Removing duplicates")

    _ = cnm.estimates.remove_duplicates(plot_duplicates=False)

    print_cnmfe_components(cnm, msg="After Remove overlapping")
    log_cnmfe_components(cnm,  msg="After Remove overlapping")

coordinates = cm.utils.visualization.get_contours(cnm.estimates.A, bg.shape, thr=.2, thr_method="max")

## Visualise Components location

Visualise the location of good and bad components on the cn_filt image and the residual optic flow image

In [5]:
# Visualise good/bad components over the residual optic flow image

optflow_img_path = os.path.join(metadata['fld'], metadata['outputfld'], 'residual_opticflow_raw.npy')
optflow_img = np.load(optflow_img_path)

f, axarr = plt.subplots(figsize=(15, 10), ncols=2)
for ax, only in zip(axarr, ["good", "bad"]):
    img = plot_components_over_image(optflow_img, ax, coordinates, 2, cnm.estimates.idx_components, cnm.estimates.idx_components_bad, cmap="viridis", only=only)
    add_colorbar_to_img(img, ax, f)

axarr[0].set(title="GOOD components")
axarr[1].set(title="BAD components")
clean_axes(f)

img_filepath = os.path.join(fld, "components_not_curated_on_optflow")
save_figure(f, img_filepath)
logging.info(f"Saved image with contours at {img_filepath}")


2020-04-23 17:08:28 PM - WARNING - MainProcess _qt.py:187 - Attribute Qt::AA_EnableHighDpiScaling must be set before QCoreApplication is created.
INFO:root:Saved image with contours at D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_not_curated_on_optflow
2020-04-23 17:08:28 PM - INFO - MainProcess <ipython-input-5-160f5ffeb7b3>:17 - Saved image with contours at D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_not_curated_on_optflow
 saved figure at: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_not_curated_on_optflow


In [6]:
# Visualize components over mean projection frame
f, axarr = plt.subplots(figsize=(15, 10), ncols=2)
for ax, only in zip(axarr, ["good", "bad"]):
    img = plot_components_over_image(bg, ax, coordinates, 2, cnm.estimates.idx_components, cnm.estimates.idx_components_bad, cmap="gray", only=only)

axarr[0].set(title="GOOD components")
axarr[1].set(title="BAD components")
clean_axes(f)

img_filepath = os.path.join(fld, "components_not_curated_on_meanproj")
save_figure(f, img_filepath)
logging.info(f"Saved image with contours at {img_filepath}")

2020-04-23 17:08:29 PM - WARNING - MainProcess _qt.py:187 - Attribute Qt::AA_EnableHighDpiScaling must be set before QCoreApplication is created.
INFO:root:Saved image with contours at D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_not_curated_on_meanproj
2020-04-23 17:08:30 PM - INFO - MainProcess <ipython-input-6-a3779789a00f>:12 - Saved image with contours at D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_not_curated_on_meanproj
 saved figure at: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_not_curated_on_meanproj


## Visualize components signal
use `view_patches_bar` to view the location and the inferred trace for each component

# MANUAL COMPONENTS CURATION
First some prep...


In [7]:
merge_commands = []

def get_component_idx(component_id):
    if component_id not in [c['neuron_id'] for c in coordinates]:
        raise ValueError("Invalid component")
    else:
        return component_id - 1

def plot_again():
    # Visualize components over mean projection frame
    bg = np.max(images, axis=0)


    cnm.estimates.coordinates = cm.utils.visualization.get_contours(cnm.estimates.A, bg.shape, thr=.3, thr_method="max")
    coordinates = cnm.estimates.coordinates

    f, ax = plt.subplots(figsize=(15, 10),)

    img = plot_components_over_image(bg, ax, coordinates, 3, cnm.estimates.idx_components, cnm.estimates.idx_components_bad, cmap="gray", alpha=.4)
    clean_axes(f)

    img_filepath = os.path.join(fld, "components_curated_on_meanproj")
    save_figure(f, img_filepath)
    logging.info(f"Saved image with contours at {img_filepath}")

### merge components

Manually merge components by specifying groups of components (using the numbers you see in the images above). Note that sometimes merging good components yields a bad one, so after that you'll have to manually specify which one is good and which one is bad

In [15]:
merge_from_commands = False

if not merge_from_commands:
    to_merge = [ # specify lists of ROI indices to be merged
        [25, 21]
    ]
else:
    to_merge = np.load(os.path.join(fld, "merge_commands.npy"), allow_pickle=True)

# now merge
for mergin in to_merge:
    if not mergin: continue
    merge_commands.append(mergin)
    idxs = [get_component_idx(i) for i in mergin]
    cnm.estimates.manual_merge(components=[idxs], params=cnm.params)


print_cnmfe_components(cnm, msg="After evaluate components")
plot_again()

INFO:root:Merging components [24, 20]
2020-04-23 17:09:42 PM - INFO - MainProcess estimates.py:1275 - Merging components [24, 20]
2020-04-23 17:09:42 PM - WARNING - MainProcess _qt.py:187 - Attribute Qt::AA_EnableHighDpiScaling must be set before QCoreApplication is created.


 ***** After evaluate components
Number of total components:  37
Number of accepted components:  16
INFO:root:Saved image with contours at D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_curated_on_meanproj
2020-04-23 17:09:43 PM - INFO - MainProcess <ipython-input-7-d0c0dddd7cc8>:24 - Saved image with contours at D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_curated_on_meanproj
 saved figure at: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_curated_on_meanproj


Once happy with the mergin you can save the merge commands to file to avoid having to do it again manually if you need to

In [16]:
# Re do automated spatial classification from caiman

# Estimate
cnm.estimates.evaluate_components(images, cnm.params, dview=dview)
cnm.estimates.threshold_spatial_components(dview=dview)
print_cnmfe_components(cnm, msg="After evaluate components")

# save
np.save(os.path.join(fld, "merge_commands_raw.npy"), np.array(merge_commands))

# plot
plot_again()

INFO:root:Component evaluation in parallel
2020-04-23 17:10:13 PM - INFO - MainProcess components_evaluation.py:800 - Component evaluation in parallel
2020-04-23 17:10:16 PM - WARNING - MainProcess _qt.py:187 - Attribute Qt::AA_EnableHighDpiScaling must be set before QCoreApplication is created.
A_thr already computed. If you want to recompute set self.A_thr to None


 ***** After evaluate components
Number of total components:  37
Number of accepted components:  18
INFO:root:Saved image with contours at D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_curated_on_meanproj
2020-04-23 17:10:17 PM - INFO - MainProcess <ipython-input-7-d0c0dddd7cc8>:24 - Saved image with contours at D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_curated_on_meanproj
 saved figure at: D:\Dropbox (UCL - SWC)\Project_vgatPAG\analysis\doric\BF164p2\19JUN26\components_curated_on_meanproj


### move from bad to good and vice versa


In [ ]:
# Specify Roi indices to move from good to bad and viceversa
from_bad_to_good = [
]

from_good_to_bad = [

]

for btg in from_bad_to_good:
    logging.info(f"Moving component {btg} from bad to good")
    cnm.estimates.idx_components_bad = np.delete(cnm.estimates.idx_components_bad, np.argwhere(cnm.estimates.idx_components_bad == btg))
    cnm.estimates.idx_components = np.append(cnm.estimates.idx_components, btg)

for gtb in from_good_to_bad:
    logging.info(f"Moving component {gtb} from good to bad")
    cnm.estimates.idx_components_bad = cnm.estimates.idx_components = np.delete(cnm.estimates.idx_components, np.argwhere(cnm.estimates.idx_components == gtb))
    np.append(cnm.estimates.idx_components_bad, gtb)


plot_again()

# Save updated cnmfe model
If you're happy with the results, you can save the model for further analysis


In [17]:
new_model_path = os.path.join(fld, "cnmfe_fit_raw_curated.hdf5")
cnm.save(new_model_path)
logging.info(f"Saving updated model at {new_model_path}")

INFO:root:Key empty_merged is not saved.
2020-04-23 17:10:27 PM - INFO - MainProcess utils.py:481 - Key empty_merged is not saved.
INFO:root:A is sparse ****
2020-04-23 17:10:27 PM - INFO - MainProcess utils.py:511 - A is sparse ****
INFO:root:W is sparse ****
2020-04-23 17:10:27 PM - INFO - MainProcess utils.py:511 - W is sparse ****
INFO:root:g is an object type
2020-04-23 17:10:27 PM - INFO - MainProcess utils.py:471 - g is an object type
INFO:root:Key merged_ROIs is not saved.
2020-04-23 17:10:27 PM - INFO - MainProcess utils.py:481 - Key merged_ROIs is not saved.
INFO:root:Key coordinates is not saved.
2020-04-23 17:10:27 PM - INFO - MainProcess utils.py:481 - Key coordinates is not saved.
INFO:root:Key groups is not saved.
2020-04-23 17:10:27 PM - INFO - MainProcess utils.py:481 - Key groups is not saved.
INFO:root:A_thr is sparse ****
2020-04-23 17:10:27 PM - INFO - MainProcess utils.py:511 - A_thr is sparse ****
INFO:root:Saving updated model at D:\Dropbox (UCL - SWC)\Project_v

# Visualize components over videos and interactive plots

Use the first cell if you just want to quickly look at the data, otherwise the next cell can be used to create a video but it's quite slow

In [ ]:
from caiman.utils.visualization import view_patches_bar

plot_again()
d1, d2 = bg.shape
view_patches_bar(Yr, cnm.estimates.A, cnm.estimates.C, cnm.estimates.b, cnm.estimates.f, d1, d2, img=bg)


In [ ]:
# To quickly look at the data use this code
SHOW_GOOD = True
SHOW_BAD = False


# Get contours from data
coordinates = cm.utils.visualization.get_contours(cnm.estimates.A, cn_filter.shape, thr=.2, thr_method="max")
good_compontents = cnm.estimates.idx_components

# prep contours for plotting
good_coords = [c['coordinates'][1:-2, ::-1] for c in coordinates if c['neuron_id'] in good_compontents]
good_coords = [c[~np.isnan(c).any(axis=1)] for c in good_coords] # remove nans
good_coords = [np.vstack([c, c[0]]) for c in good_coords]

bad_coords = [c['coordinates'][1:-2, ::-1] for c in coordinates if c['neuron_id'] not in good_compontents]
bad_coords = [c[~np.isnan(c).any(axis=1)] for c in bad_coords] # remove nans
bad_coords = [np.vstack([c, c[0]]) for c in bad_coords]

# Napari viewer
v = napari.Viewer(ndisplay=2)

v.add_image(images, name="Signal", colormap="gray")

# add good components
if SHOW_GOOD:
    layer = v.add_shapes(
        good_coords,
        shape_type='path',
        edge_width=1,
        edge_color='green',
        opacity=.5,
        face_color='royalblue',
        name='GOOD',
    )

# add bad components
if SHOW_BAD:
    layer = v.add_shapes(
        bad_coords,
        shape_type='path',
        edge_width=.8,
        edge_color='red',
        opacity=.3,
        face_color='royalblue',
        name='BAD',
    )



### video maker, very slow...

In [ ]:
frames_fld = "D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\Fede\\frames" # folder where frames will be saved before creating video


tot_frames, w, h = images.shape
frames = np.arange(0, tot_frames, 10)

plt.ioff()
print(f"Creating video with {len(frames)} frames")

# Create an image for each frame and save it
if TrFalse  # <-- !! Set this as true if you want to create the video, keeping it False to avoid doing this by accident as it's slow   for n, fnum in tqdm(enumerate(frames)):
        f, ax = plt.subplots(figsize=(120, 100), dpi=5)
        plot_components_over_image(images[fnum, :, :].copy(), ax, coordinates, 17, good_compontents)
        f.savefig(os.path.join(frames_fld, f"{n}"), dpi=5)
        plt.close()
        
plt.ion()

# Stitch images into a video
""" 
Use this ffmpeg command to stitch the frames inot a video

first cd to the folder parent of the on where you saved the frames in. 
e.g. if you saved the frames here: D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\Fede\\frames
then cd to: D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\Fede

Then use:
ffmpeg -i frames\%1d.png -c:v libx264 -vf fps=10 -pix_fmt yuv420p out.mp4

the video will be saved as D:\\Dropbox (UCL - SWC)\\Project_vgatPAG\\analysis\\doric\\Fede\\out.mp4
"""